In [560]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None
import glob as glob
import math

#### Both ADNI and A4 used the free surfer recon all command that uses the desikan killany atlas (cerebral cortex into 34 regions)
##### To do: 
- make a list of predefined regions I want to look at and then just harmonize the regional names across the 2 data frames 
- looks like desikan killany atlas is 34 cortical ones but also need to check ones like brainstem etc 

#### Further cleaning: goal is to end up with a4 and adni amy_tau merged csvs with similar naming conventions for each region/col
##### The a4 data has columns with Volume_ prefix so need to drop those 
##### need to makesure that the atlas used between the 2 datasets was consistsent 
- there's also a mean unknown col need to see if that's also present in the adni one 
- also with left right from stanford coumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- need to drop stuff like Mean_non_WM_hypointensities


* additional note- see if the atlas used was consistent between A4 and adni because the naming conventions are different 
* ex mean_ctx_rh_posteriorcingulate in the TAUSUVR vs CTX_RH_PRECENTRAL_SUVR in the UCBERKLEYAV1451_PVC_8mm csv 

In [561]:
#load and analyze a4 data 
merged_a4_at_amypos = pd.read_csv('./a4/merged_a4_at_amy_pos.csv')

In [562]:
print(merged_a4_at_amypos)

           BID         update_stamp_x       ligand  centiloid  \
0    B10423472  2018-12-06 00:00:00.0  Florbetapir    80.8687   
1    B10693248  2018-12-06 00:00:00.0  Florbetapir   113.8210   
2    B10709603  2018-12-06 00:00:00.0  Florbetapir    40.5933   
3    B10904689  2018-12-06 00:00:00.0  Florbetapir   126.6360   
4    B11089276  2018-12-06 00:00:00.0  Florbetapir    35.1012   
..         ...                    ...          ...        ...   
368  B92372599  2019-04-19 00:00:00.0  Florbetapir    40.5933   
369  B97841288  2019-04-19 00:00:00.0  Florbetapir    71.7152   
370  B98032541  2019-04-19 00:00:00.0  Florbetapir    55.2389   
371  B99033399  2019-04-19 00:00:00.0  Florbetapir    53.4082   
372  B99860801  2019-04-19 00:00:00.0  Florbetapir    35.1012   

     Mean_3rd_Ventricle  Mean_4th_Ventricle  Mean_Brain_Stem  \
0              0.845577            0.864285         0.911925   
1              1.110908            0.963089         1.000662   
2              1.041388    

#### notes from a4 doumentation (stanford pipeline)
- created using freesurfer recon- all 
- seems like a4 ones all start with Mean_

In [563]:
#check how many cols have bi_ prefix 
bi_reg_count = sum(col.startswith('bi_') for col in merged_a4_at_amypos.columns)
print(bi_reg_count)
#50 bilateral regions 

50


In [564]:
import pandas as pd

filtered_columns = [col for col in merged_a4_at_amypos.columns if col.startswith('bi_')]
column_list = list(filtered_columns)

# Create a DataFrame from the column_list
a4_col_list = pd.DataFrame({'Column Names': column_list})

# Save the DataFrame to a CSV file
a4_col_list.to_csv("./a4/a4_bi_columns.csv", index=False)

In [565]:
#load and analyze adni data
merged_adni_at_amypos = pd.read_csv("./adni/merged_adni_at_amy_pos.csv")

In [566]:
print(merged_adni_at_amypos)

      RID TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  META_TEMPORAL_SUVR  \
0      31    FBP        97.0      y1     m162  2019-04-23               1.508   
1     112    FBP        89.0      y1     m150  2018-08-13               2.223   
2     120    FBP        32.0    init     m132  2017-09-13               1.671   
3     142    FBP       107.0     v41     m120  2016-04-14               1.920   
4     210    FBP        54.0    init     m138  2018-03-13               1.842   
..    ...    ...         ...     ...      ...         ...                 ...   
406  6572    FBB       115.0      y2      m24  2020-09-30               2.185   
407  6575    FBB        89.0      y2      m24  2021-01-20               1.551   
408  6580    FBB       142.0      y2      m24  2020-10-09               1.739   
409  6581    FBB        28.0      bl       bl  2018-09-21               1.484   
410  6586    FBB        40.0      y2      m24  2020-10-15               1.337   

     META_TEMPORAL_VOLUME  

In [567]:
#seems like the volume columns are named like ******_VOLUME so drop these 
#something like endswith "_VOLUME".drop()
drop_cols = [col for col in merged_adni_at_amypos.columns if col.endswith('_VOLUME')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=drop_cols)

In [568]:
#also drop choroid 
#drop adni startswith("CHOROID")
adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith('CHOROID')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)

In [569]:
#this is without the volume 
print(merged_adni_at_amypos)

      RID TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  META_TEMPORAL_SUVR  \
0      31    FBP        97.0      y1     m162  2019-04-23               1.508   
1     112    FBP        89.0      y1     m150  2018-08-13               2.223   
2     120    FBP        32.0    init     m132  2017-09-13               1.671   
3     142    FBP       107.0     v41     m120  2016-04-14               1.920   
4     210    FBP        54.0    init     m138  2018-03-13               1.842   
..    ...    ...         ...     ...      ...         ...                 ...   
406  6572    FBB       115.0      y2      m24  2020-09-30               2.185   
407  6575    FBB        89.0      y2      m24  2021-01-20               1.551   
408  6580    FBB       142.0      y2      m24  2020-10-09               1.739   
409  6581    FBB        28.0      bl       bl  2018-09-21               1.484   
410  6586    FBB        40.0      y2      m24  2020-10-15               1.337   

     CTX_ENTORHINAL_SUVR  I

#### further cleaning adni data (only bilateral (l/r hemi regions combined))
- has col names _VOLUME for the volume so drop these 
- they also have _SUVR so maybe could only keep these as a quality control check 
- the naming conventions are stuff like CTX_LH_SUPERIORFRONTAL_SUVR (there's a CTX_LH_SUPERIORFRONTAL_SUVR and CTX_RH_SUPERIORFRONTAL_SUVR for each) 
- looks like there's a CTX_SUPERIORFRONTAL_SUVR and then a CTX_LH_SUPERIORFRONTAL_SUVR and a CTX_RH_SUPERIORFRONTAL_SUVR
- so need to only keep the CTX_SUPERIORFRONTAL_SUVR ones (something like search start with CTX**) if only looking at bilateral brain regions (this correponds to the A4 naming convention "bi_* columns" )


### drop volume before doing anything else

#### Things to drop from A4
- everything starting with Volume 
- ending with 'Ventricle" 
- specific ones ending in _plexus, vessel, _Chiasm, Unknown, _hypointensities, WM, 
- after doing this its 151 rows 

In [570]:
suffixes_to_drop = ['_Ventricle', '_plexus','_vessel','_Chiasm','_hypointensities','WM','CSF','Unknown','_y','_x',"Matter","Vent"]

# Drop columns ending with the specified suffixes
for suffix in suffixes_to_drop:
    a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.endswith(suffix)]
    merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)

In [571]:
#seems like the volume columns are named like Volume_**** so drop these columns 
#something like starts with "Volume_***".drop()
a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.startswith('Volume_')]
merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)



In [572]:
merged_a4_at_amypos

,BID,ligand,centiloid,Mean_Brain_Stem,Mean_CC_Anterior,Mean_CC_Central,Mean_CC_Mid_Anterior,Mean_CC_Mid_Posterior,Mean_CC_Posterior,Mean_Left_Accumbens_area,...,bi_precentral,bi_precuneus,bi_rostralanteriorcingulate,bi_rostralmiddlefrontal,bi_superiorfrontal,bi_superiorparietal,bi_superiortemporal,bi_supramarginal,bi_temporalpole,bi_transversetemporal
0,B10423472,Florbetapir,80.8687,0.911925,0.857168,0.733042,0.748900,0.663917,0.866686,1.199217,...,0.955041,1.079061,1.170967,1.084401,1.046561,0.904519,1.056903,0.986939,1.098317,0.949163
1,B10693248,Florbetapir,113.8210,1.000662,0.908223,0.862122,0.873952,0.837348,0.964523,1.289822,...,0.958097,1.114343,1.062407,1.174429,0.979405,0.958415,1.093653,1.146316,1.118879,1.065898
2,B10709603,Florbetapir,40.5933,1.007361,0.853996,0.872356,0.826985,0.782240,0.966260,1.433114,...,0.920471,1.039841,1.029535,0.987615,0.897984,0.900524,1.055535,1.035770,1.069306,0.981632
3,B10904689,Florbetapir,126.6360,0.920387,0.971428,0.823842,0.924871,0.792009,0.918345,1.227138,...,0.912223,1.217753,1.065679,0.993498,1.011460,1.141170,1.077074,1.141837,1.087700,0.963342
4,B11089276,Florbetapir,35.1012,1.035534,0.875824,0.751278,0.736648,0.804294,0.880089,1.459700,...,1.049212,1.160123,1.158528,1.085493,1.048737,1.015478,1.092743,1.098332,1.040324,1.066399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,B92372599,Florbetapir,40.5933,0.919593,0.696205,0.754464,0.687262,0.620278,0.825615,1.338510,...,1.008765,1.071886,0.928695,1.095746,0.997771,1.130347,1.037639,1.122645,1.031807,0.859354
369,B97841288,Florbetapir,71.7152,0.837669,0.714328,0.892816,0.857996,0.848363,1.179848,1.138793,...,1.006039,1.321353,1.060927,1.168246,1.151136,1.190718,1.055261,1.113416,1.074650,0.891608
370,B98032541,Florbetapir,55.2389,1.046703,0.694822,0.912663,0.761793,0.785953,0.889976,1.027593,...,1.061295,1.384344,1.041259,1.635588,1.237379,1.178553,1.106055,1.319375,1.063425,0.829076
371,B99033399,Florbetapir,53.4082,0.940477,0.921984,0.862013,0.859376,0.818269,0.932453,1.190066,...,0.958616,1.109726,1.063435,0.921026,0.994262,0.962612,1.016614,1.041210,1.010308,1.067383


In [573]:
#save the a4 one with just tau suvr and no vent 
merged_a4_at_amypos.to_csv('./a4/merged_a4_at_amypos_novol.csv')
# save the adni one with just tau suvr and no vent 
merged_adni_at_amypos.to_csv('./adni/merged_adni_at_amy_pos_novol.csv')

#### now drop the left and right duplicates 
- also with left right from a4 stanford doumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- in adni data it's going to be "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
- for adni some regions like orbiotfrontal gyrus, there are 3: orbiotfrontal gyrus, LEFT_ORBITOFRONTAL_GYRUS, and RIGHT_ etc so just take the one without left and right 


In [574]:
#find the adni brain regions list without "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
prefixes_to_remove = ['RIGHT_', 'LEFT_', 'CTX_RH', 'CTX_LH']
for prefix in prefixes_to_remove:
    adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith(prefix)]
    merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)
#this leaves 47 cols 

In [575]:
merged_adni_at_amypos_bi = merged_adni_at_amypos
merged_adni_at_amypos_bi.to_csv("./adni/merged_adni_at_amy_pos_bi.csv")
print(merged_adni_at_amypos_bi)

      RID TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  META_TEMPORAL_SUVR  \
0      31    FBP        97.0      y1     m162  2019-04-23               1.508   
1     112    FBP        89.0      y1     m150  2018-08-13               2.223   
2     120    FBP        32.0    init     m132  2017-09-13               1.671   
3     142    FBP       107.0     v41     m120  2016-04-14               1.920   
4     210    FBP        54.0    init     m138  2018-03-13               1.842   
..    ...    ...         ...     ...      ...         ...                 ...   
406  6572    FBB       115.0      y2      m24  2020-09-30               2.185   
407  6575    FBB        89.0      y2      m24  2021-01-20               1.551   
408  6580    FBB       142.0      y2      m24  2020-10-09               1.739   
409  6581    FBB        28.0      bl       bl  2018-09-21               1.484   
410  6586    FBB        40.0      y2      m24  2020-10-15               1.337   

     CTX_ENTORHINAL_SUVR  I

In [576]:
#drop a4 columns that have the string "lh" "rh" or "Right" or "Left"
pattern = r'(lh|rh|Left|Right)'
lr_cols = merged_a4_at_amypos.filter(regex=pattern, axis=1)
merged_a4_at_amypos_bi = merged_a4_at_amypos.drop(columns=lr_cols)
print(merged_a4_at_amypos_bi)


           BID       ligand  centiloid  Mean_Brain_Stem  Mean_CC_Anterior  \
0    B10423472  Florbetapir    80.8687         0.911925          0.857168   
1    B10693248  Florbetapir   113.8210         1.000662          0.908223   
2    B10709603  Florbetapir    40.5933         1.007361          0.853996   
3    B10904689  Florbetapir   126.6360         0.920387          0.971428   
4    B11089276  Florbetapir    35.1012         1.035534          0.875824   
..         ...          ...        ...              ...               ...   
368  B92372599  Florbetapir    40.5933         0.919593          0.696205   
369  B97841288  Florbetapir    71.7152         0.837669          0.714328   
370  B98032541  Florbetapir    55.2389         1.046703          0.694822   
371  B99033399  Florbetapir    53.4082         0.940477          0.921984   
372  B99860801  Florbetapir    35.1012         0.924592          0.844704   

     Mean_CC_Central  Mean_CC_Mid_Anterior  Mean_CC_Mid_Posterior  \
0     

In [577]:
merged_a4_at_amypos_bi.to_csv("./a4/merged_a4_at_amy_pos_bi.csv")

#### make all the a4 col names uppercase 

In [578]:
merged_a4_at_amypos_bi.columns = [col.upper() for col in merged_a4_at_amypos_bi.columns]
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'MEAN_BRAIN_STEM', 'MEAN_CC_ANTERIOR',
       'MEAN_CC_CENTRAL', 'MEAN_CC_MID_ANTERIOR', 'MEAN_CC_MID_POSTERIOR',
       'MEAN_CC_POSTERIOR', 'BI_ACCUMBENS_AREA', 'BI_AMYGDALA', 'BI_CAUDATE',
       'BI_CEREBELLUM_CORTEX', 'BI_HIPPOCAMPUS', 'BI_PALLIDUM', 'BI_PUTAMEN',
       'BI_THALAMUS', 'BI_VENTRALDC', 'BI_BANKSSTS',
       'BI_CAUDALANTERIORCINGULATE', 'BI_CAUDALMIDDLEFRONTAL', 'BI_CUNEUS',
       'BI_FRONTALPOLE', 'BI_FUSIFORM', 'BI_INFERIORPARIETAL',
       'BI_INFERIORTEMPORAL', 'BI_INSULA', 'BI_ISTHMUSCINGULATE',
       'BI_LATERALOCCIPITAL', 'BI_LATERALORBITOFRONTAL', 'BI_LINGUAL',
       'BI_MEDIALORBITOFRONTAL', 'BI_MIDDLETEMPORAL', 'BI_PARACENTRAL',
       'BI_PARAHIPPOCAMPAL', 'BI_PARSOPERCULARIS', 'BI_PARSORBITALIS',
       'BI_PARSTRIANGULARIS', 'BI_PERICALCARINE', 'BI_POSTCENTRAL',
       'BI_POSTERIORCINGULATE', 'BI_PRECENTRAL', 'BI_PRECUNEUS',
       'BI_ROSTRALANTERIORCINGULATE', 'BI_ROSTRALMIDDLEFRONTAL',
       'BI_SUPERIORFRON

In [579]:
#drop the mean columns from a4
a4_mean_drop_cols = [col for col in merged_a4_at_amypos_bi.columns if col.startswith('MEAN')]
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.drop(columns=a4_mean_drop_cols)

In [580]:
#remove the BI prefix from A4 
merged_a4_at_amypos_bi.columns = merged_a4_at_amypos_bi.columns.str.replace("BI_",'')

In [581]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'FRONTALPOLE', 'FUSIFORM',
       'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA', 'ISTHMUSCINGULATE',
       'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL', 'LINGUAL',
       'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


In [582]:
merged_adni_at_amypos_bi = merged_adni_at_amypos_bi.drop(columns=['EXAMDATE','VISCODE','VISCODE2','update_stamp'])

In [583]:
#drop CTX prefix and drop SUVR suffix
merged_adni_at_amypos_bi.columns = merged_adni_at_amypos_bi.columns.str.replace("CTX_",'').str.replace('_SUVR','')

In [584]:
print(merged_adni_at_amypos_bi.columns)

Index(['RID', 'TRACER', 'CENTILOIDS', 'META_TEMPORAL', 'ENTORHINAL',
       'INFERIOR_CEREBGM', 'HEMIWM', 'BRAIN_STEM', 'MIDDLEFR', 'ORBITOFR',
       'PARSFR', 'BANKSSTS', 'CAUDALANTERIORCINGULATE', 'CUNEUS', 'FUSIFORM',
       'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA', 'ISTHMUSCINGULATE',
       'LATERALOCCIPITAL', 'LINGUAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PERICALCARINE', 'POSTCENTRAL', 'POSTERIORCINGULATE',
       'PRECENTRAL', 'PRECUNEUS', 'ROSTRALANTERIORCINGULATE',
       'SUPERIORFRONTAL', 'SUPERIORPARIETAL', 'SUPERIORTEMPORAL',
       'SUPRAMARGINAL', 'TEMPORALPOLE', 'TRANSVERSETEMPORAL', 'ACCUMBENS_AREA',
       'AMYGDALA', 'CAUDATE', 'PALLIDUM', 'PUTAMEN', 'THALAMUS_PROPER'],
      dtype='object')


In [585]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'FRONTALPOLE', 'FUSIFORM',
       'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA', 'ISTHMUSCINGULATE',
       'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL', 'LINGUAL',
       'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


#### now find over lapping regions (with same name)

In [586]:
#rename BID to rid in a4 to make it consistent with adni 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'BID':'RID'})
#rename ligand col to tracer 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'LIGAND':'TRACER'})
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'CENTILOID':'CENTILOIDS'})


In [587]:
#get a list of regions in both dataframes (30 common regions)
common_regions = list(set(merged_adni_at_amypos_bi.columns)& set(merged_a4_at_amypos_bi.columns))
# print(len(common_regions))
#now filter dfs based on common regions 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi[common_regions]
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi[common_regions]
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.set_index('RID')
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi_harm.set_index('RID')


In [588]:
#round to 3 decimal points so its like the adni 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.round(3)
print(merged_a4_at_amypos_bi_harm)

           TRANSVERSETEMPORAL  SUPERIORTEMPORAL  INFERIORTEMPORAL  FUSIFORM  \
RID                                                                           
B10423472               0.949             1.057             1.226     1.188   
B10693248               1.066             1.094             1.300     1.242   
B10709603               0.982             1.056             1.184     1.235   
B10904689               0.963             1.077             1.349     1.244   
B11089276               1.066             1.093             1.158     1.182   
...                       ...               ...               ...       ...   
B92372599               0.859             1.038             1.249     1.142   
B97841288               0.892             1.055             1.405     1.256   
B98032541               0.829             1.106             1.346     1.202   
B99033399               1.067             1.017             1.037     1.057   
B99860801               0.929             0.928     

In [589]:
print(merged_adni_at_amypos_bi_harm)

      TRANSVERSETEMPORAL  SUPERIORTEMPORAL  INFERIORTEMPORAL  FUSIFORM  \
RID                                                                      
31                 1.484             1.502             1.532     1.295   
112                1.746             2.003             2.480     2.015   
120                1.209             1.575             1.733     1.494   
142                1.136             1.726             1.960     1.446   
210                1.298             1.661             1.977     1.606   
...                  ...               ...               ...       ...   
6572               0.808             1.926             2.434     1.792   
6575               1.215             1.400             1.574     1.400   
6580               1.230             1.636             1.857     1.494   
6581               0.997             1.378             1.613     1.284   
6586               1.028             1.208             1.399     1.225   

      POSTERIORCINGULATE  INSULA  PER

In [590]:
#save the new csvs that have the same regions across both 
merged_a4_at_amypos_bi_harm.to_csv('./a4/merged_a4_at_amy_pos_bi_harm.csv')
merged_adni_at_amypos_bi_harm.to_csv('./adni/merged_adni_at_amy_pos_bi_harm.csv')

I think I might just start with a list of brain regions I know will be implicated in AD 
1. entorhinal (a4 bi_entorhinal, adni CTX_ENTORHINAL)
2. hippocampus (a4 bi_hippocampus, adni CTX_HIPPOCAMPUS) 
